<a href="https://colab.research.google.com/github/Sai0472003/PatentLitigation_Impact/blob/main/Causal_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [78]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Basic Data Loading and Cleaning

In [79]:
#loading pre-cleaned csv files
clean_plaintiff_RnD = pd.read_csv('/content/drive/My Drive/ECO225_Final_Project/DATA/clean_plaintiff_RnD.csv')
clean_plaintiff_RnD = clean_plaintiff_RnD[['case_number', 'filing_quarter_rnd',
                                           'closing_quarter_rnd', 'delta_rnd_sales', 'minus_one_rnd',
                                           'minus_two_rnd', 'minus_three_rnd',
                                           'plus_one_rnd', 'plus_two_rnd',
                                           'plus_three_rnd', 'state_province',
                                           'case_length','materials', 'industrials',
                                           'consumer_discretionary', 'info_tech',
                                           'health_care']]

clean_defendent_RnD = pd.read_csv('/content/drive/My Drive/ECO225_Final_Project/DATA/clean_defendent_RnD.csv')
clean_defendent_RnD = clean_defendent_RnD[['case_number', 'filing_quarter_rnd',
                                           'closing_quarter_rnd','delta_rnd_sales', 'minus_one_rnd',
                                           'minus_two_rnd', 'minus_three_rnd',
                                           'plus_one_rnd', 'plus_two_rnd',
                                           'plus_three_rnd', 'state_province',
                                           'case_length','materials', 'industrials',
                                           'consumer_discretionary', 'info_tech',
                                           'health_care']]



clean_plaintiff_npm = pd.read_csv('/content/drive/My Drive/ECO225_Final_Project/DATA/clean_plaintiff_npm.csv')
clean_plaintiff_npm = clean_plaintiff_npm[['case_number','filing_quarter_npm',
                                           'closing_quarter_npm', 'delta_npm', 'minus_one_npm',
                                           'minus_two_npm', 'minus_three_npm',
                                           'plus_one_npm', 'plus_two_npm',
                                           'plus_three_npm', 'state_province',
                                           'case_length','materials', 'industrials',
                                           'consumer_discretionary', 'info_tech',
                                           'health_care']]

clean_defendent_npm = pd.read_csv('/content/drive/My Drive/ECO225_Final_Project/DATA/clean_defendent_npm.csv')
clean_defendent_npm = clean_defendent_npm[['case_number', 'filing_quarter_npm',
                                           'closing_quarter_npm', 'delta_npm', 'minus_one_npm',
                                           'minus_two_npm', 'minus_three_npm',
                                           'plus_one_npm', 'plus_two_npm',
                                           'plus_three_npm', 'state_province',
                                           'case_length','materials', 'industrials',
                                           'consumer_discretionary', 'info_tech',
                                           'health_care']]


pd.set_option('display.max_columns', None)

In [80]:
#adding defendant/plaintiff dummies and cocat merging into single df
clean_plaintiff_npm['Plaintiff'] = 1
clean_defendent_npm['Defendant'] = 1

npm = pd.concat([clean_defendent_npm, clean_plaintiff_npm], axis=0)
npm['Defendant'] = npm['Defendant'].fillna(0)
npm['Plaintiff'] = npm['Plaintiff'].fillna(0)


clean_plaintiff_RnD['Plaintiff'] = 1
clean_defendent_RnD['Defendant'] = 1

rnd = pd.concat([clean_defendent_RnD, clean_plaintiff_RnD], axis=0)
rnd['Defendant'] = rnd['Defendant'].fillna(0)
rnd['Plaintiff'] = rnd['Plaintiff'].fillna(0)

In [81]:
JFC_data = pd.read_csv('/content/drive/My Drive/ECO225_Final_Project/DATA/without_dummies_cases.csv')

In [82]:
JFC_data['jurisdictional_basis'].value_counts()

,count
jurisdictional_basis,
Federal Question,53523
U.S. Government Defendant,238
U.S. Government Plaintiff,117


In [83]:
JFC_data = JFC_data[['case_number', 'case_name', 'DEF', 'PLT', 'jurisdictional_basis',
                     'CIRCUIT', 'DISTRICT', 'OFFICE', 'JURY', 'NOS', 'CLASSACT',
                     'COUNTY', 'PROCPROG', 'DISP', 'NOJ', 'JUDGMENT']]

JFC_data

,case_number,case_name,DEF,PLT,jurisdictional_basis,CIRCUIT,DISTRICT,OFFICE,JURY,NOS,CLASSACT,COUNTY,PROCPROG,DISP,NOJ,JUDGMENT
0,1:98-cv-00596,Mettke v. Touchnet Information,TOUCHNET INFORMATION,METTKE,Federal Question,11,26,1,-8,830,0,1015,3,14,-8,-8
1,1:98-cv-00654,"Monsanto Co. Inc. v. Styron, et al","STYRON, ET AL",MONSANTO CO. INC.,Federal Question,11,28,1,-8,830,0,1097,10,13,-8,-8
2,1:18-cv-00012,Sundesa LLC v. Eurark LLC,EURARK LLC,SUNDESA LLC,Federal Question,8,60,1,B,830,0,88888,3,13,0,0
3,2:99-cv-00674,"Star Video Duplictns v. US Magnetics Corp, et al","US MAGNETICS CORP, ET AL",STAR VIDEO DUPLICTNS,Federal Question,9,70,2,-8,830,0,4013,10,13,-8,-8
4,2:99-cv-02348,"Brother Industries v. Jing Mold Enterprise, et al","JING MOLD ENTERPRISE, ET AL",BROTHER INDUSTRIES,Federal Question,9,73,2,-8,830,0,99999,3,13,-8,-8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53873,2:99-cv-00216,"Impact Roller Tech, et al v. Star Aggregates Inc",STAR AGGREGATES INC,"IMPACT ROLLER TECH, ET AL",Federal Question,10,89,2,-8,830,0,88888,5,5,0,1
53874,2:93-cv-00004,Construction Tech v. Heating,HEATING,CONSTRUCTION TECH,Federal Question,10,89,2,-8,830,0,88888,3,12,-8,-8
53875,2:95-cv-00217,Wyoming Woolens v. Acorn Products Co,ACORN PRODUCTS CO,WYOMING WOOLENS,Federal Question,10,89,2,-8,830,0,56039,1,12,-8,-8
53876,2:97-cv-00019,Chey Land & Cattle v. Crown Cork & Seal Co,CROWN CORK & SEAL CO,CHEY LAND & CATTLE,Federal Question,10,89,2,-8,830,0,56021,1,12,-8,-8


In [84]:
npm = npm.merge(JFC_data, on='case_number', how='left')
rnd = rnd.merge(JFC_data, on='case_number', how='left')

In [85]:
npm.head()

,case_number,filing_quarter_npm,closing_quarter_npm,delta_npm,minus_one_npm,minus_two_npm,minus_three_npm,plus_one_npm,plus_two_npm,plus_three_npm,state_province,case_length,materials,industrials,consumer_discretionary,info_tech,health_care,Defendant,Plaintiff,case_name,DEF,PLT,jurisdictional_basis,CIRCUIT,DISTRICT,OFFICE,JURY,NOS,CLASSACT,COUNTY,PROCPROG,DISP,NOJ,JUDGMENT
0,2:11-cv-01037,0.163247,0.226309,0.063062,0.208424,0.222403,0.228644,0.228330,0.227446,0.229692,TX,1210,0,0,0,0,1,1.0,0.0,Carroll v. Texas Instruments Inc.(MAG+),TEXAS INSTRUMENTS INC.,CARROLL,Federal Question,11,27,2,B,830,0,1123,10,14,0,0
1,1:15-cv-01059,0.244910,0.796015,0.551105,-0.038337,0.002570,0.177977,0.783359,-0.105780,-0.160819,CA,691,0,0,1,0,0,1.0,0.0,Blazer v. Ebay Inc,EBAY INC,BLAZER,Federal Question,11,26,1,B,830,0,1029,10,13,0,0
2,5:11-cv-01349,0.051063,0.047465,-0.003598,0.051389,0.052812,0.056058,0.066061,0.066351,0.069411,MO,516,0,0,1,0,0,1.0,0.0,"Homtex, Inc v. Leggett & Platt, Inc et al","LEGGETT & PLATT, INC, ET AL","HOMTEX, INC",Federal Question,11,26,5,P,830,0,1043,10,13,0,0
3,2:87-cv-00979,0.010817,0.037153,0.026336,0.011557,0.016528,0.014094,0.034018,0.029158,0.023642,TX,775,0,1,0,0,0,1.0,0.0,Thrall Car Manufactu v. Trinity Industries,TRINITY INDUSTRIES INC,THRALL CAR MFG CO,Federal Question,11,26,2,-8,830,0,88888,5,13,-8,-8
4,1:17-cv-00179,0.208723,0.210924,0.002201,0.207433,0.207305,0.212408,0.211246,0.215499,0.220046,CA,149,0,0,0,0,1,1.0,0.0,"Pen-One Acquisition Group, LLC v. Apple Inc.",APPLE INC.,"PEN-ONE ACQUISITION GROUP, LLC",Federal Question,11,28,1,B,830,0,1097,3,12,0,0


In [86]:
rnd.head()

,case_number,filing_quarter_rnd,closing_quarter_rnd,delta_rnd_sales,minus_one_rnd,minus_two_rnd,minus_three_rnd,plus_one_rnd,plus_two_rnd,plus_three_rnd,state_province,case_length,materials,industrials,consumer_discretionary,info_tech,health_care,Defendant,Plaintiff,case_name,DEF,PLT,jurisdictional_basis,CIRCUIT,DISTRICT,OFFICE,JURY,NOS,CLASSACT,COUNTY,PROCPROG,DISP,NOJ,JUDGMENT
0,2:11-cv-01037,0.125210,0.100666,-0.024544,0.118179,0.117259,0.114605,0.098920,0.098242,0.098462,TX,1210,0,0,0,0,1,1.0,0.0,Carroll v. Texas Instruments Inc.(MAG+),TEXAS INSTRUMENTS INC.,CARROLL,Federal Question,11,27,2,B,830,0,1123,10,14,0,0
1,1:15-cv-01059,0.108168,0.125455,0.017288,0.110958,0.111719,0.000000,0.124701,0.127940,0.125441,CA,691,0,0,1,0,0,1.0,0.0,Blazer v. Ebay Inc,EBAY INC,BLAZER,Federal Question,11,26,1,B,830,0,1029,10,13,0,0
2,5:11-cv-01349,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.005375,0.005953,0.006165,MO,516,0,0,1,0,0,1.0,0.0,"Homtex, Inc v. Leggett & Platt, Inc et al","LEGGETT & PLATT, INC, ET AL","HOMTEX, INC",Federal Question,11,26,5,P,830,0,1043,10,13,0,0
3,2:87-cv-00979,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,TX,775,0,1,0,0,0,1.0,0.0,Thrall Car Manufactu v. Trinity Industries,TRINITY INDUSTRIES INC,THRALL CAR MFG CO,Federal Question,11,26,2,-8,830,0,88888,5,13,-8,-8
4,1:17-cv-00179,0.049904,0.050520,0.000616,0.048885,0.048194,0.046701,0.050661,0.051407,0.052867,CA,149,0,0,0,0,1,1.0,0.0,"Pen-One Acquisition Group, LLC v. Apple Inc.",APPLE INC.,"PEN-ONE ACQUISITION GROUP, LLC",Federal Question,11,28,1,B,830,0,1097,3,12,0,0


# Creating Useful Dummies for the JFC Categorical Data

In [87]:
#using PROCPROG to create disp_before_issue_joined
npm['disp_before_issue_joined'] = 0
rnd['disp_before_issue_joined'] = 0

npm.loc[npm['PROCPROG'].isin([1,2,11,12]), 'disp_before_issue_joined'] = 1
rnd.loc[rnd['PROCPROG'].isin([1,2,11,12]), 'disp_before_issue_joined'] = 1

npm = npm.drop(columns=['PROCPROG'])
rnd = rnd.drop(columns=['PROCPROG'])

In [94]:
#using DISP to create dummies for dissmissals, final judgment and case transfers

#For the npm df
npm['disp_dismissed'] = 0
npm['disp_judgment'] = 0
npm['disp_transfer'] = 0

npm.loc[npm['DISP'].isin([0,1,10,11]), 'disp_transfer'] = 1
npm.loc[npm['DISP'].isin([2,3,12,13,14]), 'disp_dismissed'] = 1
npm.loc[npm['DISP'].isin([4,5,6,7,8,9,15,16,17,18,19,20]), 'disp_judgment'] = 1

#extra dummies for dismissals with settlement or voluntary withdrawl by plaintiff
npm['dismissed_settlement'] = 0
npm['dismissed_voluntary'] = 0

npm.loc[npm['DISP'].isin([13]), 'dismissed_settlement'] = 1
npm.loc[npm['DISP'].isin([12]), 'dismissed_voluntary'] = 1


npm = npm.drop(columns=['DISP'])

#For the rnd df
rnd['disp_dismissed'] = 0
rnd['disp_judgment'] = 0
rnd['disp_transfer'] = 0

rnd.loc[rnd['DISP'].isin([0,1,10,11]), 'disp_transfer'] = 1
rnd.loc[rnd['DISP'].isin([2,3,12,13,14]), 'disp_dismissed'] = 1
rnd.loc[rnd['DISP'].isin([4,5,6,7,8,9,15,16,17,18,19,20]), 'disp_judgment'] = 1

#extra dummies for dismissals with settlement or voluntary withdrawl by plaintiff
rnd['dismissed_settlement'] = 0
rnd['dismissed_voluntary'] = 0

rnd.loc[rnd['DISP'].isin([13]), 'dismissed_settlement'] = 1
rnd.loc[rnd['DISP'].isin([12]), 'dismissed_voluntary'] = 1

rnd = rnd.drop(columns=['DISP'])

In [98]:
rnd.groupby('disp_judgment')['JUDGMENT'].value_counts()

disp_judgment  JUDGMENT
0               0          2450
               -8           834
1               2           229
                0           201
                1           182
                3           105
               -8            27
                4            26
Name: count, dtype: int64

In [93]:
#using JUDGMENT to create dummies for who one the cases that went to judgment
npm['plaintiff_won'] = 1
npm['']

,case_number,filing_quarter_rnd,closing_quarter_rnd,delta_rnd_sales,minus_one_rnd,minus_two_rnd,minus_three_rnd,plus_one_rnd,plus_two_rnd,plus_three_rnd,state_province,case_length,materials,industrials,consumer_discretionary,info_tech,health_care,Defendant,Plaintiff,case_name,DEF,PLT,jurisdictional_basis,CIRCUIT,DISTRICT,OFFICE,JURY,NOS,CLASSACT,COUNTY,DISP,NOJ,JUDGMENT,disp_before_issue_joined,disp_dismissed,disp_judgment,disp_transfer,dismissed_settlement,dismissed_voluntary
0,2:11-cv-01037,0.125210,0.100666,-0.024544,0.118179,0.117259,0.114605,0.098920,0.098242,0.098462,TX,1210,0,0,0,0,1,1.0,0.0,Carroll v. Texas Instruments Inc.(MAG+),TEXAS INSTRUMENTS INC.,CARROLL,Federal Question,11,27,2,B,830,0,1123,14,0,0,0,1,0,0,0,0
1,1:15-cv-01059,0.108168,0.125455,0.017288,0.110958,0.111719,0.000000,0.124701,0.127940,0.125441,CA,691,0,0,1,0,0,1.0,0.0,Blazer v. Ebay Inc,EBAY INC,BLAZER,Federal Question,11,26,1,B,830,0,1029,13,0,0,0,1,0,0,1,0
2,5:11-cv-01349,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.005375,0.005953,0.006165,MO,516,0,0,1,0,0,1.0,0.0,"Homtex, Inc v. Leggett & Platt, Inc et al","LEGGETT & PLATT, INC, ET AL","HOMTEX, INC",Federal Question,11,26,5,P,830,0,1043,13,0,0,0,1,0,0,1,0
3,2:87-cv-00979,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,TX,775,0,1,0,0,0,1.0,0.0,Thrall Car Manufactu v. Trinity Industries,TRINITY INDUSTRIES INC,THRALL CAR MFG CO,Federal Question,11,26,2,-8,830,0,88888,13,-8,-8,0,1,0,0,1,0
4,1:17-cv-00179,0.049904,0.050520,0.000616,0.048885,0.048194,0.046701,0.050661,0.051407,0.052867,CA,149,0,0,0,0,1,1.0,0.0,"Pen-One Acquisition Group, LLC v. Apple Inc.",APPLE INC.,"PEN-ONE ACQUISITION GROUP, LLC",Federal Question,11,28,1,B,830,0,1097,12,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4049,3:97-cv-00860,0.083717,0.089442,0.005725,0.079640,0.069746,0.068435,0.095037,0.095022,0.092048,WI,84,0,0,0,1,0,0.0,1.0,"LUNAR CORPORATION v. OEC MEDICAL SYSTEMS, INC.","OEC MEDICAL SYSTEMS, INC.",LUNAR CORPORATION,Federal Question,7,58,3,-8,830,0,55025,13,-8,-8,1,1,0,0,1,0
4050,1:15-cv-00149,0.104989,0.092344,-0.012646,0.109238,0.107514,0.114665,0.109108,0.131259,0.147169,CA,63,0,0,0,1,0,0.0,1.0,"GILEAD SCIENCES, INC. et al v. Mylan Inc. et al","MYLAN INC., ET AL","GILEAD SCIENCES, INC., ET AL",Federal Question,4,24,1,N,830,0,54061,13,0,0,0,1,0,0,1,0
4051,1:19-cv-00097,0.154950,0.169681,0.014731,0.153549,0.153224,0.147265,0.177769,0.196337,0.228701,NY,157,0,0,0,1,0,0.0,1.0,Pfizer Inc. et al v. Mylan Pharmaceuticals Inc...,"MYLAN PHARMACEUTICALS I, ET AL","PFIZER INC., ET AL",Federal Question,4,24,1,N,830,0,88888,10,0,0,0,0,0,1,0,0
4052,1:20-cv-00244,0.245896,0.199822,-0.046074,0.228701,0.196337,0.177769,0.163436,0.170124,0.156567,NY,236,0,0,0,1,0,0.0,1.0,Pfizer Inc. et al v. Mylan Pharmaceuticals Inc...,"MYLAN PHARMACEUTICALS I, ET AL","PFIZER INC., ET AL",Federal Question,4,24,1,N,830,0,88888,0,0,0,0,0,0,1,0,0
